In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten , GlobalAveragePooling2D , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# Set paths and image size
train_path = r'C:\\Users\\zakja\\datasets\\skin diseases\\train'
test_path = r'C:\\Users\\zakja\\datasets\\skin diseases\\test'
IMAGE_SIZE = [224, 224]
folders = glob(r'C:\\Users\\zakja\\datasets\\skin diseases\\test\\*')
# Load pre-trained ResNet50 model
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=IMAGE_SIZE+[3])

# Freeze layers in pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Add new layers to the pre-trained model
x = GlobalAveragePooling2D()(resnet.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=resnet.input, outputs=prediction)

# Compile the model
optimizer = Adam(lr=0.001)  # Adjust learning rate if necessary
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define data generators for training and testing with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_set = train_datagen.flow_from_directory(
    train_path, target_size=IMAGE_SIZE, batch_size=128, class_mode='categorical'
)
test_set = test_datagen.flow_from_directory(
    test_path, target_size=IMAGE_SIZE, batch_size=128, class_mode='categorical'
)

# Apply early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=50,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks=[early_stopping]
)

F:\anaconda\envs\emb\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 15597 images belonging to 23 classes.
Found 4003 images belonging to 23 classes.
Epoch 1/50
 81/122 [==================>...........] - ETA: 34s - loss: 2.7811 - accuracy: 0.2235